In [1]:
import pytorch_lightning as pl
import segmentation_models_pytorch as smp
import torch
import random
import os
import torchmetrics

import pandas as pd
import torch.nn.functional as F
import numpy as np

from torchvision import transforms as py_transforms
from torch.utils.data import Dataset, DataLoader
from lovasz import lovasz_hinge
from albumentations import *
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [40]:

sz = 256

def getBatchSize(sz):

    # 48 for 512
    if sz == 256:
        bs = 64
    elif sz == 512:
        bs = 48 #batch size
    elif sz == 1024:
        bs = 12 #batch size
    else:
        bs = 12
        
    return bs

def getPaths(sz):
    TRAIN = "./data/{}x{}/train/".format(sz, sz)
    MASKS = "./data/{}x{}/masks/".format(sz, sz)
    # TRAIN = "./kaggle/data/{}x{}/train/".format(sz, sz)
    # MASKS = "./data/{}x{}/masks/".format(sz, sz)
    LABELS = "./hubmap-kidney-segmentation/train.csv"
    
    return TRAIN, MASKS, LABELS

# 12 for 1024
# Input Parameters
nfolds = 4
fold = 0
SEED = 2020
NUM_WORKERS = 16


def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # torch.backends.
    torch.backends.cudnn.deterministic = True
    # the following line gives ~10% speedup
    # but may lead to some stochasticity in the results
    torch.backends.cudnn.benchmark = True


seed_everything(SEED)

def getStats(sz):

    # https://www.kaggle.com/iafoss/256x256-images
    if sz == 256:
        mean = np.array([0.62950801, 0.45658695, 0.67636472])
        std = np.array([0.15937661, 0.21920461, 0.14160065])

    # 512x512
    elif sz == 512:
        mean = np.array([0.64246083, 0.48809628, 0.68370845])
        std = np.array([0.17817486, 0.24006252, 0.16265069])

    # 1024x1024
    else:
        mean = np.array([0.63244577, 0.49794695, 0.66810544])
        std = np.array([0.22345657, 0.26747085, 0.21520419])
        
    return mean, std


def img2tensor(img, dtype: np.dtype = np.float32):
    if img.ndim == 2: img = np.expand_dims(img, 2)
    img = np.transpose(img, (2, 0, 1))
    return torch.from_numpy(img.astype(dtype, copy=False)).to(dtype=torch.float16)


In [41]:

class HuBMAPDataset(Dataset):
    def __init__(self, mean, std, train_path, mask_path, label_path, fold=fold, train=True, tfms=None, cache=True):
        self.TRAIN = train_path
        self.LABELS = label_path
        self.MASKS = mask_path
        
        
        ids = pd.read_csv(self.LABELS).id.values
        kf = KFold(n_splits=nfolds, random_state=SEED, shuffle=True)
        ids = set(ids[list(kf.split(ids))[fold][0 if train else 1]])
        self.fnames = [fname for fname in os.listdir(self.TRAIN) if fname.split('_')[0] in ids]
        self.train = train
        self.tfms = tfms
        self.cache = cache
        
        self.mean = mean
        self.std = std

        if self.cache:
            self.images = [cv2.cvtColor(cv2.imread(os.path.join(self.TRAIN, fname)), cv2.COLOR_BGR2RGB)
                             for fname in tqdm(self.fnames)]
            self.masks = [cv2.imread(os.path.join(self.MASKS, fname), cv2.IMREAD_GRAYSCALE)
                              for fname in tqdm(self.fnames)]

    def __len__(self):
        return len(self.fnames)

    def __getitem__(self, idx):
        fname = self.fnames[idx]

        if self.cache:
            img = self.images[idx]
            mask = self.masks[idx]
        else:
            img = cv2.cvtColor(cv2.imread(os.path.join(self.TRAIN, fname)), cv2.COLOR_BGR2RGB)
            mask = cv2.imread(os.path.join(self.MASKS, fname), cv2.IMREAD_GRAYSCALE)

        if self.tfms is not None:
            augmented = self.tfms(image=img, mask=mask)
            img, mask = augmented['image'], augmented['mask']
        return py_transforms.ToTensor()(img), py_transforms.ToTensor()(mask)


In [42]:
def get_aug(p=1.0):
    return Compose([
        HorizontalFlip(),
        VerticalFlip(),
        RandomRotate90(),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=15, p=0.9,
                         border_mode=cv2.BORDER_REFLECT),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.3),
        OneOf([
            HueSaturationValue(10, 15, 10),
            CLAHE(clip_limit=2),
            RandomBrightnessContrast(),
        ], p=0.3),
    ], p=p)


In [43]:
def createDataset(size, folds, fold, train, tfms=None):
    mean, std = getStats(size)
    
    train_path, mask_path, label_path = getPaths(size)
    
    ds = HuBMAPDataset(mean=mean, std=std, train_path=train_path, mask_path=mask_path, label_path=label_path,
                       fold=fold, tfms=tfms, cache=False, train=train)
    
    return ds


def createDataLoaders(size, folds, fold):
    
    ds_train = createDataset(size=size, folds=folds, fold=fold, train=True, tfms=get_aug())
    ds_val = createDataset(size=size, folds=folds, fold=fold, train=False)
    
    bs = getBatchSize(size)
    
    train_loader = DataLoader(ds_train, batch_size=bs, num_workers=20)
    val_loader = DataLoader(ds_val, batch_size=bs, num_workers=20)
    
    return train_loader, val_loader

createDataset(256, 4, False, 0)

createDataLoaders(256, 4, 0)

(<torch.utils.data.dataloader.DataLoader at 0x7f70adeaeb38>,
 <torch.utils.data.dataloader.DataLoader at 0x7f70adeaeac8>)

In [44]:
class HubmapSystem(pl.LightningModule):
    
    def __init__(self, learning_rate=1.3182567385564074e-07):
        super().__init__()
        
        self.learning_rate = learning_rate
        
#         print("Learning Rate: ", self.learning_rate)
        
        self.num_classes = 1
        
        self.epoch_dice = []
        
        
#         aux_params=dict(
#             pooling='avg',             # one of 'avg', 'max'
#             dropout=0.5,               # dropout ratio, default is None
#             activation='sigmoid',      # activation function, default is None
#             classes=self.num_classes,                 # define number of output labels
#         )
        
        self.model = smp.FPN(
            encoder_name="efficientnet-b7",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
            encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
            in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
            classes=self.num_classes,                      # model output channels (number of classes in your dataset)
            decoder_dropout=0.1,
        )
        
#         self.model.to(dtype=torch.float16)
        
#         if self.precision == 16:
            
#             self.model.to(dtype=torch.float16)
#         else:
#             self.model.to(dtype=torch.float32)
        
#         for name, param in self.model.named_parameters():
#             param.requires_grad = False
            
#         for name, param in self.model.segmentation_head.named_parameters():
#             param.requires_grad = True
        
#         for name, parameter in self.model.named_parameters():
#             print(name, parameter.requires_grad)
    
    def forward(self, x):
        res = self.model(x)
        return res
    
    def training_step(self, batch, batch_nb):
        # REQUIRED
        x, y = batch
        y_hat = self(x)
#         print(y_hat)
        loss = self.symmetric_lovasz(y_hat, y)
        self.log("train_loss", loss)
        self.log("dice", self.dice_metric(y_hat, y), prog_bar=True)
        
        self.epoch_dice.append(self.dice_metric(y_hat, y).item())
        
#         print(torchmetrics.functional.dice_score(y_hat, y))
        return loss

    def on_train_epoch_end(self, outputs):
        avg = np.mean(self.epoch_dice)
        self.log("MEAN DICE", avg, prog_bar=True)
        self.epoch_dice = []

    def validation_step(self, batch, batch_nb):
        # OPTIONAL
        x, y = batch
        y_hat = self(x)
        print("yhat:", y_hat)
        print("y", y)
        loss = self.symmetric_lovasz(y_hat, y)
        print(loss)
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_dice", self.dice_metric(y_hat, y), prog_bar=True)
        
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
        return optimizer
        
    def symmetric_lovasz(self, outputs, targets):
        return 0.5*(lovasz_hinge(outputs, targets) + lovasz_hinge(-outputs, 1.0 - targets))
    
    def dice_metric(self, y_hat, y):
        return torchmetrics.functional.f1(y_hat, y, num_classes=self.num_classes) 

In [45]:
#ds_train = HuBMAPDataset(fold=0, train=True, tfms=get_aug(), cache=False)
#ds_val = HuBMAPDataset(fold=0, train=False, cache=False)

In [46]:
#train_loader = DataLoader(ds_train, batch_size=bs, num_workers=16)
#val_loader = DataLoader(ds_val, batch_size=bs, num_workers=16)

In [47]:

#system = HubmapSystem()
#trainer = pl.Trainer(gpus=1, precision=16, deterministic=True,
#                     benchmark=True, check_val_every_n_epoch=1, accumulate_grad_batches=22)
#trainer.fit(system, train_loader, val_loader)


In [48]:
for fold in range(nfolds):
    print("FOLD: ", fold)
    
#     ds_train = HuBMAPDataset(fold=fold, train=True, tfms=get_aug(), cache=False)
#     ds_val = HuBMAPDataset(fold=fold, train=False, cache=False)
    
#     train_loader = DataLoader(ds_train, batch_size=bs, num_workers=20)
#     val_loader = DataLoader(ds_val, batch_size=bs, num_workers=20)
    
    early_stop_callback = EarlyStopping(
       monitor='val_loss',
       min_delta=0.00,
       patience=10,
       verbose=False,
       mode='min'
    )
    
    system = HubmapSystem.load_from_checkpoint("eff_{}.ckpt".format(fold))
    system.learning_rate = 0.005
    trainer = pl.Trainer(gpus=1, precision=16, deterministic=True,callbacks=[early_stop_callback],
                     benchmark=True, check_val_every_n_epoch=1, accumulate_grad_batches=4)
    
#     # Run learning rate finder
#     lr_finder = trainer.tuner.lr_find(system, train_loader, val_loader)

#     # Results can be found in
#     lr_finder.results

#     # Plot with
#     fig = lr_finder.plot(suggest=True)
#     fig.show()
    
#     system.lr = lr_finder.suggestion()
#     print("LR: ", system.lr)

    
    #256
    train_loader, val_loader = createDataLoaders(256, 4, fold)
    
    trainer.fit(system, train_loader, val_loader)
    
    #512
    train_loader, val_loader = createDataLoaders(512, 4, fold)
    trainer.fit(system, train_loader, val_loader)
    
    #1024
    train_loader, val_loader = createDataLoaders(1024, 4, fold)
    trainer.fit(system, train_loader, val_loader)
    
    trainer.save_checkpoint(f'eff_{fold}.ckpt')
    

FOLD:  0


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.

  | Name  | Type | Params
-------------------------------
0 | model | FPN  | 65.7 M
-------------------------------
65.7 M    Trainable params
0         Non-trainable params
65.7 M    Total params
262.663   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

yhat: tensor([[[[-2.6562, -2.9238, -3.1895,  ..., -4.7109, -4.4414, -4.1719],
          [-2.7402, -2.9941, -3.2480,  ..., -4.8672, -4.6016, -4.3320],
          [-2.8223, -3.0645, -3.3066,  ..., -5.0234, -4.7578, -4.4922],
          ...,
          [-4.4141, -4.6836, -4.9492,  ..., -4.8828, -5.0000, -5.1133],
          [-4.3008, -4.5586, -4.8164,  ..., -4.8008, -4.9258, -5.0547],
          [-4.1875, -4.4375, -4.6836,  ..., -4.7188, -4.8555, -4.9961]]],


        [[[-2.8672, -3.0703, -3.2734,  ..., -5.8594, -5.5898, -5.3203],
          [-2.8906, -3.0977, -3.3066,  ..., -6.1406, -5.9180, -5.6953],
          [-2.9160, -3.1270, -3.3379,  ..., -6.4219, -6.2461, -6.0703],
          ...,
          [-5.0742, -5.3789, -5.6836,  ..., -0.5928, -0.6260, -0.6587],
          [-4.8711, -5.1562, -5.4375,  ..., -0.6318, -0.5850, -0.5381],
          [-4.6680, -4.9297, -5.1953,  ..., -0.6704, -0.5435, -0.4170]]],


        [[[ 0.2467,  0.4019,  0.5571,  ..., -3.9785, -4.0586, -4.1367],
          [ 0.2256, 

Training: 0it [00:00, ?it/s]

ValueError: Attempting to unscale FP16 gradients.

In [ ]:
system.model.eval()

t = torch.rand([256, 256])

system(t)

In [33]:
v = torch.arange(0, 64)
stacked = torch.stack([v]*(256**2)).T
print(v)
print(stacked)

res = stacked.view([64, 256, 256])
print(res.shape)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63])
tensor([[ 0,  0,  0,  ...,  0,  0,  0],
        [ 1,  1,  1,  ...,  1,  1,  1],
        [ 2,  2,  2,  ...,  2,  2,  2],
        ...,
        [61, 61, 61,  ..., 61, 61, 61],
        [62, 62, 62,  ..., 62, 62, 62],
        [63, 63, 63,  ..., 63, 63, 63]])
torch.Size([64, 256, 256])
